In [1]:
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn import metrics
from contextlib import redirect_stdout
from itertools import compress
import scipy.sparse as sparse
import lightgbm as lgb
import pandas as pd
import numpy as np
import tqdm
import os
import gc
import sys
sys.path.append('../code/pipeline/')
sys.path.append('../code/utils/')
sys.path.append('../code/')
import data_pipeline as dp
import data_utils as du
import perf_utils as pu
import eval_utils as eu
import io_utils as iu
import config

In [2]:
user_one_feat_names = config.USER_SINGLE_FEAT_NAMES
user_multi_feat_names = config.USER_MULTI_FEAT_NAMES
ad_feat_names = config.AD_FEAT_NAMES

In [3]:
# ================
# Data Preparation
# ================
# instantiate data loader
emb_loaders = (dp.EmbeddingDataManager.build_data(feat_name, "max", 1) 
               for feat_name in user_multi_feat_names)
union_loader = dp.DataUnion(*emb_loaders)

with pu.profiler("loading training data"):
    cols_emb_train, X_emb_tv = union_loader.load("train")
    # X_emb_tv = sparse.csr_matrix(X_emb_tv)
    gc.collect()

print("Train Data Shape (Embedding): {}".format(X_emb_tv.shape))
print("Train Column Numbers (Embedding): {}".format(len(cols_emb_train)))

[04:59:39] Finish loading training data. △M: +15.21GB. △T: 2.3 minutes.
Train Data Shape (Embedding): (8798814, 464)
Train Column Numbers (Embedding): 464


In [4]:
df_train = du.load_raw_data("train")
y = df_train['label'].values.copy()
y = (y + 1) / 2  # -1, 1 -> 0, 1

In [5]:
# n_splits = 3
# sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=1 / 3, random_state=20180505)  # for reproducibility
# split_indices = [(train_index, valid_index) for train_index, valid_index in sss.split(df_train, y)]

n_splits = 5  # use 3 instead of 5 to save time
skf = StratifiedKFold(n_splits=n_splits)
split_indices = [(train_index, valid_index) for train_index, valid_index in skf.split(df_train, y)]

In [6]:
aids = df_train['aid'].values
with pu.profiler("splitting train/valid set"):
    train_index, valid_index = split_indices[0]
    # X_train, X_valid = X_tv[train_index, :], X_tv[valid_index, :]
    X_train, X_valid = X_emb_tv[train_index, :], X_emb_tv[valid_index, :]
    y_train, y_valid = y[train_index], y[valid_index]
    aids_train, aids_valid = aids[train_index], aids[valid_index]
    # assert X_train.shape[0] + X_valid.shape[0] == X_tv.shape[0]
    assert X_train.shape[0] + X_valid.shape[0] == X_emb_tv.shape[0]
    
    # del X_tv
    del X_emb_tv
    gc.collect()

print("Training Set Size: {}".format(X_train.shape))
print("Validation Set Size: {}".format(X_valid.shape))

[04:59:58] Finish splitting train/valid set. △M: +120.85MB. △T: 14.2 seconds.
Training Set Size: (7039050, 464)
Validation Set Size: (1759764, 464)


In [7]:
with pu.profiler("preparing LightGBM data"):
    # lgb_train = lgb.Dataset(X_train.astype(np.float32), y_train, feature_name=cols_train)  # cause bugs I don't know how to fix
    # lgb_valid = lgb.Dataset(X_valid.astype(np.float32), y_valid, feature_name=cols_train)
    lgb_train = lgb.Dataset(X_train.astype(np.float32), y_train)
    lgb_valid = lgb.Dataset(X_valid.astype(np.float32), y_valid)
    gc.collect()

[05:00:10] Finish preparing LightGBM data. △M: +15.21GB. △T: 12.0 seconds.


In [8]:
with pu.profiler("releasing memory"):
    del X_train
    # del X_valid
    gc.collect()

[05:00:11] Finish releasing memory. △M: -12.17GB. △T: 1.5 seconds.


In [9]:
log_folder = os.path.join(config.LOG_DIR, 'lgbm/pipeline/0518/')
log_file = 'v3.log'
log_path = os.path.join(log_folder, log_file)
os.makedirs(log_folder, exist_ok=True)

In [10]:
# v2 parameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': 15,
    'num_leaves': 120,
    'learning_rate': 0.15,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'verbose': 0
}
num_rounds = 1000
# # v3 parameters
# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': 'auc',
#     'max_depth': 6,
#     'num_leaves': 64,
#     'learning_rate': 0.1,
#     'feature_fraction': 0.5,
#     'bagging_fraction': 0.5,
#     'verbose': 0
# }
# num_rounds = 5000
eval_auc = eu.build_lightgbm_online_auc_eval(aids_train, aids_valid)

with iu.DuplicatedLogger(log_path):
    lgbm = lgb.train(params,
                     lgb_train,
                     num_boost_round=num_rounds,
                     valid_sets=[lgb_train, lgb_valid], 
                     valid_names=['train', 'valid1'],
                     feval = eval_auc,
                     early_stopping_rounds=50)

[1]	train's auc: 0.628924	train's online_auc: 0.653838	valid1's auc: 0.62695	valid1's online_auc: 0.650055
Training until validation scores don't improve for 50 rounds.
[2]	train's auc: 0.635773	train's online_auc: 0.660355	valid1's auc: 0.633197	valid1's online_auc: 0.656405
[3]	train's auc: 0.638669	train's online_auc: 0.663429	valid1's auc: 0.636172	valid1's online_auc: 0.659767
[4]	train's auc: 0.640653	train's online_auc: 0.665812	valid1's auc: 0.638401	valid1's online_auc: 0.661912
[5]	train's auc: 0.64115	train's online_auc: 0.666102	valid1's auc: 0.638988	valid1's online_auc: 0.662288
[6]	train's auc: 0.643203	train's online_auc: 0.66807	valid1's auc: 0.640731	valid1's online_auc: 0.663629
[7]	train's auc: 0.644448	train's online_auc: 0.669326	valid1's auc: 0.641864	valid1's online_auc: 0.664948
[8]	train's auc: 0.645186	train's online_auc: 0.6697	valid1's auc: 0.642787	valid1's online_auc: 0.665776
[9]	train's auc: 0.646328	train's online_auc: 0.671044	valid1's auc: 0.643786	v

[77]	train's auc: 0.689139	train's online_auc: 0.709224	valid1's auc: 0.667727	valid1's online_auc: 0.68658
[78]	train's auc: 0.689597	train's online_auc: 0.70961	valid1's auc: 0.667822	valid1's online_auc: 0.686682
[79]	train's auc: 0.689957	train's online_auc: 0.709991	valid1's auc: 0.667869	valid1's online_auc: 0.686739
[80]	train's auc: 0.690402	train's online_auc: 0.71042	valid1's auc: 0.667948	valid1's online_auc: 0.686992
[81]	train's auc: 0.690798	train's online_auc: 0.710772	valid1's auc: 0.667965	valid1's online_auc: 0.687093
[82]	train's auc: 0.691068	train's online_auc: 0.711062	valid1's auc: 0.668028	valid1's online_auc: 0.687184
[83]	train's auc: 0.69144	train's online_auc: 0.711389	valid1's auc: 0.668115	valid1's online_auc: 0.687225
[84]	train's auc: 0.69185	train's online_auc: 0.711727	valid1's auc: 0.668114	valid1's online_auc: 0.687199
[85]	train's auc: 0.69222	train's online_auc: 0.711994	valid1's auc: 0.668194	valid1's online_auc: 0.687198
[86]	train's auc: 0.69261

[152]	train's auc: 0.714787	train's online_auc: 0.733922	valid1's auc: 0.667799	valid1's online_auc: 0.68639
[153]	train's auc: 0.715119	train's online_auc: 0.734222	valid1's auc: 0.667761	valid1's online_auc: 0.686392
[154]	train's auc: 0.715495	train's online_auc: 0.73458	valid1's auc: 0.667712	valid1's online_auc: 0.686334
Early stopping, best iteration is:
[104]	train's auc: 0.698948	train's online_auc: 0.718633	valid1's auc: 0.668356	valid1's online_auc: 0.687209


In [11]:
# these will cause LightGBM bug I don't know how to fix. Just skip it
log_file = 'v3.feature_importance.csv'
log_path = os.path.join(log_folder, log_file)

# df_feature_importance = pd.DataFrame({"feature": cols_train, "importance": lgbm.feature_importance()})
df_feature_importance = pd.DataFrame({"feature": cols_emb_train, "importance": lgbm.feature_importance()})
df_feature_importance = df_feature_importance.sort_values("importance", ascending=False)
df_feature_importance.to_csv(log_path, index=False)
df_feature_importance.head(30)

,feature,importance
176,kw2_embedding_49,189
175,kw2_embedding_48,159
159,kw2_embedding_32,147
132,kw2_embedding_5,140
147,kw2_embedding_20,138
322,topic2_embedding_45,133
127,kw2_embedding_0,129
138,kw2_embedding_11,127
156,kw2_embedding_29,122
146,kw2_embedding_19,116


In [16]:
with pu.profiler("preparing valid set"):
    cols_emb_train, X_emb_tv = union_loader.load("train")
    X_valid = X_emb_tv[valid_index, :]
    del X_emb_tv
    gc.collect()

[02:31:13] Finish preparing valid set. △M: +3.06GB. △T: 2.4 minutes.


In [17]:
with pu.profiler("making prediction on validation set"):
    df_valid = df_train.iloc[valid_index]
    proba_valid = lgbm.predict(X_valid.astype(np.float32))

log_file = 'v3.online_auc.csv'
log_path = os.path.join(log_folder, log_file)
df_score = eu.online_auc(df_valid['aid'], y_valid, proba_valid, ret_verbose=True)
df_score.rename(columns={'selector': 'aid'}, inplace=True)
df_score = df_score[['aid', 'auc']]  # sort columns
df_score = df_score.sort_values("auc", ascending=False)
df_score.to_csv(log_path, index=False)

[02:31:30] Finish making prediction on validation set. △M: +12.0KB. △T: 7.0 seconds.


In [18]:
online_auc = df_score['auc'].mean()
simple_auc = metrics.roc_auc_score(y_valid, proba_valid)
print("Online AUC: {:.6f}".format(online_auc))
print("Simple AUC: {:.6f}".format(simple_auc))

Online AUC: 0.685241
Simple AUC: 0.666967


In [19]:
with pu.profiler("cleaning memory"):
    del lgb_train
    del lgb_valid
    # del X_train
    del X_valid
    gc.collect()

[02:35:07] Finish cleaning memory. △M: -6.19GB. △T: 0.7 seconds.


In [17]:
with pu.profiler("loading testing data"):
    cols_test, X_test = union_loader.load("test1")
    X_test = sparse.csr_matrix(X_test)
    gc.collect()

print("Test Data Shape: {}".format(X_test.shape))
# print("Test Column Numbers: {}".format(len(cols_test)))

[15:23:44] Finish loading testing data. △M: +1.6GB. △T: 1.2 minutes.
[15:24:12] Finish trimming testing set. △M: -1.22GB. △T: 27.6 seconds.
Test Data Shape (After masking): (2265989, 1791142)


In [18]:
df_test = du.load_raw_data("test")
X_test = X_test.astype(np.float32)

with pu.profiler("making prediction on testing set"):
    proba_test = lgbm.predict(X_test)
    assert len(proba_test.shape) == 1
    assert proba_test.shape[0] == df_test.shape[0]

[15:25:56] Finish making prediction on testing set. △M: -5.14GB. △T: 1.5 minutes.


In [19]:
subm_folder = '../subm/lgbm/0518_v3'
subm_file = 'submission.csv'
subm_path = os.path.join(subm_folder, subm_file)
os.makedirs(subm_folder, exist_ok=True)

subm = df_test.copy()
subm["score"] = proba_test
subm.to_csv(subm_path, index=False)